<a href="https://colab.research.google.com/github/mann-brinson/valkyrie_interview/blob/main/step2_pyspark/Freq_Itemsets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount Google Drive to access data files
from google.colab import drive, files

#Load the order_products_train_many.json input file
drive.mount('/content/gdrive')
files.upload()

In [ ]:
#Paramters
input_file = "order_products_train_many.json"
output_freq_itemsets = "freq_itemsets.json"
output_item_baskets = "item_baskets.json"

In [ ]:
#Install pyspark dependencies on Google Colab
import os
import json

# Install dependencies: jdk-8, hadoop-2.7, spark-3.1.1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
#Run a local spark session
import findspark
findspark.init()

#Apriori algorithm library
from pyspark.ml.fpm import FPGrowth

from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Valkyrie") \
    .getOrCreate()

sc = spark.sparkContext

**Compute frequent itemsets**

In [ ]:
#Load the json file of orders containing order_products purchased by users who made at least 20 orders
jsonDF = spark.read.json(input_file)

In [ ]:
#Call the FPGrowth library

#The minimum support for an itemset to be identified as frequent. 
#For example, if an item appears 3 out of 5 transactions, it has a support of 3/5=0.6.
minSupport = 0.001

#minimum confidence for generating Association Rule. Confidence is an indication of how often an association rule 
# has been found to be true. For example, if in the transactions itemset X appears 4 times, X and Y co-occur only 2 times, 
# the confidence for the rule X => Y is then 2/4 = 0.5. The parameter will not affect the mining for frequent itemsets, 
# but specify the minimum confidence for generating association rules from frequent itemsets.
minConfidence = 0

fpGrowth = FPGrowth(itemsCol="basket_content", minSupport=minSupport, minConfidence=minConfidence)
model = fpGrowth.fit(jsonDF)

# Display frequent itemsets.
model.freqItemsets.show(10)

+--------------------+----+
|               items|freq|
+--------------------+----+
|             [36695]| 295|
|      [36695, 40706]|  41|
|       [36695, 5876]|  34|
|      [36695, 21137]|  77|
|[36695, 21137, 13...|  33|
|      [36695, 47626]|  45|
|      [36695, 47209]|  82|
|[36695, 47209, 13...|  46|
|      [36695, 27966]|  60|
|      [36695, 13176]|  97|
+--------------------+----+
only showing top 10 rows



In [ ]:
#Convert pyspark.sql.dataframe to pandas.dataframe
pandasDF = model.freqItemsets.toPandas()

#Convert from dataframe to dict
freq_items_dict = pandasDF.to_dict('records')
print(freq_items_dict)

#Add the length attribute and then serialize as json 
freq_items_dict2 = dict()
f_itemset_id = 1
for freq_itemset in freq_items_dict:
  len_itemset = len(freq_itemset['items'])
  freq_items_dict2[f_itemset_id] = {'length': len_itemset, 'freq': freq_itemset['freq'], 'items': freq_itemset['items']}
  f_itemset_id += 1

#Save the result to a json file
with open(output_freq_itemsets, 'w') as outfile:
    json.dump(freq_items_dict2, outfile)

In [ ]:
#Download the resulting json file, then upload to DynamoDB
#TODO: Configure DynamoDB Access Key in Google Colab, to put the JSON file in DynamoDB

In [ ]:
#Create an item_baskets dict
#For each item, in each itemset, generate a value for that item-key
item_baskets = dict()
for f_itemset_id in freq_items_dict2.keys():
  for item in freq_items_dict2[f_itemset_id]["items"]:
    if item not in item_baskets:
      item_baskets[item] = {'itemset_ids': [f_itemset_id]}
    else:
      item_baskets[item]['itemset_ids'].append(f_itemset_id)
  
#Save the result to json file
with open(output_item_baskets, 'w') as outfile:
    json.dump(item_baskets, outfile)

In [ ]:
#Download the resulting json file, then upload to DynamoDB
#TODO: Configure DynamoDB Access Key in Google Colab, to put the JSON file in DynamoDB